In [ ]:
# Ячейка 1: Импорты и базовые настройки
from dotenv import load_dotenv
load_dotenv()
import os
import ast
import copy
import gc
import itertools
import joblib
import json
import math
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import pandas as pd
import pickle
import random
import re
import scipy as sp
import string
import sys
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import roc_auc_score, classification_report, f1_score, confusion_matrix, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

from tqdm.auto import tqdm

import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

warnings.filterwarnings("ignore")

# Установка устройства
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current device is: {device}")

In [ ]:
# Ячейка 2: Пути к файлам и конфигурация
main_dir = os.getenv("MAIN_DIR")
data_dir = os.path.join(main_dir, "liar2")
extra_data_dir = os.path.join(main_dir, "liar-twitter")
output_dir = os.path.join(main_dir, "output")
os.makedirs(output_dir, exist_ok=True)

class config:
    MODEL = "microsoft/deberta-v3-xsmall"
    MAX_LEN = 128
    BATCH_SIZE_TRAIN = 32
    BATCH_SIZE_VALID = 32
    EPOCHS = 20
    LEARNING_RATE = 2e-5
    LEARNING_RATE_BIN = 4e-6
    SEED = 42
    NUM_CLASSES = 4  # меньше на 1, т.к. далее мы объединяем 0 и 1 классы
    NUM_CLASSES_BIN = 1  # для бинарной модели
    NUM_WORKERS = 0  # если мало памяти, лучше 0
    GRADIENT_ACCUMULATION_STEPS = 4
    WEIGHT_DECAY = 0.01
    WEIGHT_DECAY_BIN = 1e-4

In [ ]:
# Ячейка 3: Функция для установки seed
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(config.SEED)

In [ ]:
# Ячейка 4.1: Загрузка данных liar2 (https://huggingface.co/datasets/chengxuphd/liar2)
train_df = pd.read_csv(os.path.join(data_dir, "train.csv"))
valid_df = pd.read_csv(os.path.join(data_dir, "valid.csv"))
test_df = pd.read_csv(os.path.join(data_dir, "test.csv"))

print(f"Train shape liar2: {train_df.shape}")
print(f"Valid shape liar2: {valid_df.shape}")
print(f"Test shape liar2: {test_df.shape}")
display(train_df.head())

In [ ]:
# Ячейка 4.2: Загрузка данных liar-twitter (https://www.kaggle.com/datasets/muhammadimran112233/liar-twitter-dataset/data)
extra_df = pd.read_csv(os.path.join(extra_data_dir, "Liar_Dataset.csv"))

label_mapping = {"pants-fire": 0, "FALSE": 1, "barely-true": 2, "half-true": 3, "mostly-true": 4, "TRUE": 5}
extra_df['label'] = extra_df['label'].map(label_mapping)

extra_train_df, extra_test_valid_df = train_test_split(
    extra_df,
    test_size=0.2,
    stratify=extra_df['label'],
    random_state=config.SEED
)

extra_valid_df, extra_test_df = train_test_split(
    extra_test_valid_df,
    test_size=0.5,
    stratify=extra_test_valid_df['label'],
    random_state=config.SEED
)

print(f"Train shape liar-twitter: {extra_train_df.shape}")
print(f"Valid shape liar-twitter: {extra_valid_df.shape}")
print(f"Test shape liar-twitter: {extra_test_df.shape}")
display(extra_df.head())

In [ ]:
# Ячейка 4.3: Объединение датасетов
train_df = train_df[['statement', 'label']]
valid_df = valid_df[['statement', 'label']]
test_df = test_df[['statement', 'label']]

extra_train_df = extra_train_df[['statement', 'label']]
extra_valid_df = extra_valid_df[['statement', 'label']]
extra_test_df = extra_test_df[['statement', 'label']]

train_df = pd.concat([train_df, extra_train_df], ignore_index=True)
valid_df = pd.concat([valid_df, extra_valid_df], ignore_index=True)
test_df = pd.concat([test_df, extra_test_df], ignore_index=True)

print(f"Train shape: {train_df.shape}")
print(train_df['label'].value_counts())
print(f"Valid shape: {valid_df.shape}")
print(f"Test shape: {test_df.shape}")

In [ ]:
# Ячейка 5.1: Изменение данных

# Объединяем 0 (pants-on-fire) и 1 (false) классы.
# Модель их плохо различает + они оба по сути ложные.
# Эти классы описывают лишь наглость лжи и без
# дополнительного контекста их будет сложно различить.
train_df['label'] = train_df['label'].replace({0:1})
valid_df['label'] = valid_df['label'].replace({0:1})
test_df['label'] = test_df['label'].replace({0:1})

# Теперь сдвигаем все классы на -1, чтобы классы начинались с 0
train_df['label'] = train_df['label'] - 1
valid_df['label'] = valid_df['label'] - 1
test_df['label'] = test_df['label'] - 1

In [ ]:
# Ячейка 5.2: Балансировка классов
# Андерсемплинг класса 0 до 20% от общего размера
majority_class = 0
max_samples = train_df['label'].value_counts().sort_values().iloc[len(train_df['label'].unique()) - 2]

df_majority = train_df[train_df['label'] == majority_class]
df_minority = train_df[train_df['label'] != majority_class]

df_majority_downsampled = resample(df_majority,
                                   replace=False,
                                   n_samples=max_samples,
                                   random_state=config.SEED)

train_df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Оверсемплинг остальных классов до 15% от общего размера
min_samples = round(sum(train_df['label'].value_counts())*0.15)

dfs = []
for label in train_df_balanced['label'].unique():
    df_class = train_df_balanced[train_df_balanced['label'] == label]
    if len(df_class) < min_samples:
        df_upsampled = resample(df_class,
                                replace=True,
                                n_samples=min_samples,
                                random_state=config.SEED)
        dfs.append(df_upsampled)
    else:
        dfs.append(df_class)

train_df_balanced = pd.concat(dfs).sample(frac=1, random_state=config.SEED).reset_index(drop=True)

print("Баланс классов до ресэмплинга:")
print(train_df['label'].value_counts())

print()
print(f"Андерсемплинг до: {max_samples}")
print(f"Оверсемплинг до: {min_samples}")
print()

print("Баланс классов после ресэмплинга:")
print(train_df_balanced['label'].value_counts())

In [ ]:
# Ячейка 5.3: Выделение данных под бинарную модель и балансировка
train_df_balanced_bin = train_df_balanced.copy()
train_df_balanced_bin['label_bin'] = train_df_balanced['label'].apply(lambda x: 1 if x == 4 else 0 if x == 0 else 2)
train_df_balanced_bin = train_df_balanced_bin[train_df_balanced_bin['label_bin'] != 2].reset_index(drop=True)
train_df_balanced_bin.drop(columns=['label'])
valid_df_bin = valid_df.copy()
valid_df_bin['label_bin'] = valid_df['label'].apply(lambda x: 1 if x == 4 else 0 if x == 0 else 2)
valid_df_bin = valid_df_bin[valid_df_bin['label_bin'] != 2].reset_index(drop=True)
valid_df_bin.drop(columns=['label'])
test_df_bin = test_df.copy()
test_df_bin['label_bin'] = test_df['label'].apply(lambda x: 1 if x == 4 else 0 if x == 0 else 2)
test_df_bin = test_df_bin[test_df_bin['label_bin'] != 2].reset_index(drop=True)
test_df_bin.drop(columns=['label'])

# Делаем андерсемпл неправды для бинарной модели
# df_class_0 = train_df_balanced_bin[train_df_balanced_bin['label_bin'] == 0]
# df_class_1 = train_df_balanced_bin[train_df_balanced_bin['label_bin'] == 1]

# n_samples = len(df_class_0)

# df_class_1_downsampled = resample(df_class_1,
#                                  replace=False,
#                                  n_samples=n_samples,
#                                  random_state=config.SEED)

# train_df_balanced_bin = pd.concat([df_class_0, df_class_1_downsampled]).reset_index(drop=True)

# Для вероятностной модели фильтруем только неправду
train_df_balanced_multi = train_df_balanced[train_df_balanced['label'] != 4].copy()
valid_df_multi = valid_df[valid_df['label'] != 4].copy()

In [ ]:
# Ячейка 6: Токенизатор
tokenizer = AutoTokenizer.from_pretrained(config.MODEL)

In [ ]:
# Ячейка 7: Dataset для многоклассовой классификации
class LiarDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, label_column='label'):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_column = label_column

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row['statement']
        labels = row[self.label_column]
        inputs = self.tokenizer(text, max_length=self.max_len, padding='max_length', truncation=True, return_tensors="pt")
        item = {key: val.squeeze(0) for key, val in inputs.items()}
        item['labels'] = torch.tensor(labels, dtype=torch.long)
        return item

In [ ]:
# Ячейка 8.1: Создаем датасеты и загрузчики бинарной модели
train_dataset_bin = LiarDataset(train_df_balanced_bin, tokenizer, config.MAX_LEN, label_column='label_bin')
valid_dataset_bin = LiarDataset(valid_df_bin, tokenizer, config.MAX_LEN, label_column='label_bin')
test_dataset_bin = LiarDataset(test_df_bin, tokenizer, config.MAX_LEN, label_column='label_bin')

train_loader_bin = DataLoader(train_dataset_bin, batch_size=config.BATCH_SIZE_TRAIN, shuffle=True, num_workers=config.NUM_WORKERS)
valid_loader_bin = DataLoader(valid_dataset_bin, batch_size=config.BATCH_SIZE_VALID, shuffle=False, num_workers=config.NUM_WORKERS)
test_loader_bin = DataLoader(test_dataset_bin, batch_size=config.BATCH_SIZE_VALID, shuffle=False, num_workers=config.NUM_WORKERS)

In [ ]:
# Ячейка 8.2: Создаем датасеты и загрузчики вероятностной модели
train_dataset_multi = LiarDataset(train_df_balanced_multi, tokenizer, config.MAX_LEN, label_column='label')
valid_dataset_multi = LiarDataset(valid_df_multi, tokenizer, config.MAX_LEN, label_column='label')

train_loader_multi = DataLoader(train_dataset_multi, batch_size=config.BATCH_SIZE_TRAIN, shuffle=True, num_workers=config.NUM_WORKERS)
valid_loader_multi = DataLoader(valid_dataset_multi, batch_size=config.BATCH_SIZE_VALID, shuffle=False, num_workers=config.NUM_WORKERS)

In [ ]:
# Ячейка 9: Модель с выходом для любого количества классов
class CustomModel(nn.Module):
    def __init__(self, model_name, num_classes, dropout_rate):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.config.output_hidden_states = False
        self.model = AutoModel.from_pretrained(model_name, config=self.config)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:,0]  # CLS токен
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [ ]:
# Ячейка 10: Функции обучения и валидации
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    losses = []
    preds = []
    targets = []

    for batch in tqdm(dataloader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        preds.append(outputs.argmax(dim=1).detach().cpu().numpy())
        targets.append(labels.detach().cpu().numpy())

    avg_loss = np.mean(losses)
    preds = np.concatenate(preds)
    targets = np.concatenate(targets)
    acc = (preds == targets).mean()
    return avg_loss, acc

def valid_epoch(model, dataloader, criterion):
    model.eval()
    losses = []
    preds = []
    targets = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)

            losses.append(loss.item())
            preds.append(outputs.argmax(dim=1).detach().cpu().numpy())
            targets.append(labels.detach().cpu().numpy())

    avg_loss = np.mean(losses)
    preds = np.concatenate(preds)
    targets = np.concatenate(targets)
    acc = (preds == targets).mean()
    return avg_loss, acc

In [ ]:
# Ячейка 11: EarlyStopping
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
            return False
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                return True
        else:
            self.best_score = score
            self.counter = 0
            return False

In [ ]:
# Ячейка 12.1: Бинарная классификация Правда/Ложь
# Ctrl+F8 - выполнить все ячейки выше

model_bin = CustomModel(config.MODEL, config.NUM_CLASSES_BIN, 0.3).to(device)
#model_bin.load_state_dict(torch.load(os.path.join(output_dir, "deberta_v3_xsmall_binary_liar.pth")))

optimizer = torch.optim.AdamW(model_bin.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY_BIN)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

# Дисбаланс классов
class_counts = train_df_balanced_bin['label_bin'].value_counts()
pos_weight = class_counts[0] / class_counts[1]
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight).to(device))

scaler = torch.cuda.amp.GradScaler()

truth_threshold = 0.3  # начальный порог, будет подбираться
early_stopping = EarlyStopping(patience=3, min_delta=1e-4)
epochs_no_improve = 0
best_macro_f1 = 0
best_threshold = truth_threshold

looping = True

for epoch in range(config.EPOCHS):
    model_bin.train()
    train_losses = []
    train_preds = []
    train_targets = []

    optimizer.zero_grad(set_to_none=True)

    for step, batch in enumerate(tqdm(train_loader_bin, desc=f"Training Epoch {epoch+1}")):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).float().unsqueeze(1)

        with torch.cuda.amp.autocast():
            outputs = model_bin(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss = loss / config.GRADIENT_ACCUMULATION_STEPS

        scaler.scale(loss).backward()

        if (step + 1) % config.GRADIENT_ACCUMULATION_STEPS == 0 or (step + 1) == len(train_loader_bin):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)

        train_losses.append(loss.item() * config.GRADIENT_ACCUMULATION_STEPS)
        probs = torch.sigmoid(outputs).detach().cpu()
        preds = (probs > truth_threshold).long().numpy()
        train_preds.extend(preds)
        train_targets.extend(labels.detach().cpu().long().numpy())

    train_acc = np.mean(np.array(train_preds) == np.array(train_targets))
    train_loss = np.mean(train_losses)

    # Валидация
    model_bin.eval()
    val_losses = []
    val_probs = []
    val_targets = []

    with torch.no_grad():
        for batch in tqdm(valid_loader_bin, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device).float().unsqueeze(1)

            outputs = model_bin(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            val_losses.append(loss.item())

            probs = torch.sigmoid(outputs).cpu().numpy()
            val_probs.extend(probs.flatten())
            val_targets.extend(labels.cpu().numpy().flatten())

    val_loss = np.mean(val_losses)
    val_targets_np = np.array(val_targets)

    # Подбор оптимального порога по F1 на валидации
    from sklearn.metrics import precision_recall_curve

    precision, recall, thresholds = precision_recall_curve(val_targets_np, val_probs)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
    best_idx = np.argmax(f1_scores)
    best_threshold_epoch = thresholds[best_idx]
    best_f1_epoch = f1_scores[best_idx]

    # Классификация с оптимальным порогом
    val_preds = (np.array(val_probs) > best_threshold_epoch).astype(int)

    val_acc = (val_preds == val_targets_np).mean()
    macro_f1 = f1_score(val_targets_np, val_preds, average='macro')

    print(f"Epoch {epoch + 1}/{config.EPOCHS} | Train loss: {train_loss:.4f} acc: {train_acc:.4f} | Val loss: {val_loss:.4f} acc: {val_acc:.4f} macro F1: {macro_f1:.4f}")
    print(f"Best threshold this epoch: {best_threshold_epoch:.3f} with macro F1: {best_f1_epoch:.4f}")
    print("Classification report (macro):")
    print(classification_report(val_targets_np, val_preds, digits=4, zero_division=0))
    print("Confusion matrix:")
    print(confusion_matrix(val_targets_np, val_preds))
    print("")

    scheduler.step(val_acc)

    # Ранняя остановка по макро F1
    if macro_f1 > best_macro_f1 + 1e-4:
        best_macro_f1 = macro_f1
        best_threshold = best_threshold_epoch
        epochs_no_improve = 0
        # Сохраняем лучшую модель
        torch.save({
            'model_state_dict': model_bin.state_dict(),
            'best_threshold': best_threshold
        }, os.path.join(output_dir, "deberta_v3_xsmall_liar_binary_best_with_threshold.pth"))
        print("Сохранили лучшую модель и порог.")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= early_stopping.patience:
            print(f"Ранняя остановка на эпохе {epoch+1}")
            break

    # user_input = input("Хотите продолжить тренировку? (yes/no): ").strip().lower()
    # if user_input not in ('yes', 'y'):
    #     print("Остановка по запросу пользователя.")
    #     break

print(f"Обучение завершено. Лучший macro F1: {best_macro_f1:.4f} при пороге {best_threshold:.3f}")

In [ ]:
# Ячейка 12.2: Вероятностная классификация степени лжи
model = CustomModel(config.MODEL, config.NUM_CLASSES, 0.5).to(device)
#model.load_state_dict(torch.load(os.path.join(output_dir, "deberta_v3_xsmall_probability_liar.pth")))

optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY)

# Дисбаланс классов
class_counts = train_df_balanced['label'].value_counts().sort_index()
class_weights = 1.0 / torch.tensor(class_counts.values, dtype=torch.float)
class_weights = class_weights / class_weights.sum()
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

scaler = torch.cuda.amp.GradScaler()

looping = True

while looping:
  for epoch in range(config.EPOCHS):
      model.train()
      train_losses = []
      train_preds = []
      train_targets = []

      optimizer.zero_grad()

      for step, batch in enumerate(tqdm(train_loader_multi, desc=f"Training Epoch {epoch+1}")):
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['labels'].to(device)

          with torch.cuda.amp.autocast():
              outputs = model(input_ids, attention_mask)
              loss = criterion(outputs, labels)
              loss = loss / config.GRADIENT_ACCUMULATION_STEPS

          scaler.scale(loss).backward()

          if (step + 1) % config.GRADIENT_ACCUMULATION_STEPS == 0 or (step + 1) == len(train_loader_multi):
              scaler.step(optimizer)
              scaler.update()
              optimizer.zero_grad()

          train_losses.append(loss.item() * config.GRADIENT_ACCUMULATION_STEPS)
          preds = outputs.argmax(dim=1).detach().cpu().numpy()
          train_preds.extend(preds)
          train_targets.extend(labels.detach().cpu().numpy())

      train_acc = np.mean(np.array(train_preds) == np.array(train_targets))
      train_loss = np.mean(train_losses)

      # Валидация (без градиентов)
      model.eval()
      val_losses = []
      val_preds = []
      val_targets = []

      with torch.no_grad():
          for batch in tqdm(valid_loader_multi, desc="Validation"):
              input_ids = batch['input_ids'].to(device)
              attention_mask = batch['attention_mask'].to(device)
              labels = batch['labels'].to(device)

              outputs = model(input_ids, attention_mask)
              loss = criterion(outputs, labels)

              val_losses.append(loss.item())
              preds = outputs.argmax(dim=1).detach().cpu().numpy()
              val_preds.extend(preds)
              val_targets.extend(labels.detach().cpu().numpy())

      val_acc = np.mean(np.array(val_preds) == np.array(val_targets))
      val_loss = np.mean(val_losses)

      print(f"Epoch {epoch+1}/{config.EPOCHS} | Train loss: {train_loss:.4f} acc: {train_acc:.4f} | Val loss: {val_loss:.4f} acc: {val_acc:.4f}")
      print("Classification report (macro):")
      print(classification_report(val_targets, val_preds, digits=4, zero_division=0))
      print("Macro F1-score:", f1_score(val_targets, val_preds, average='macro'))
      print("Confusion matrix:")
      print(confusion_matrix(val_targets, val_preds))
      print("")

  user_input = input("Хотите продолжить тренировку? (yes/no): ").strip().upper()
  if "YES" in user_input or "Y" in user_input:
      looping = True
  else:
      looping = False

# Сохранение модели
torch.save(model.state_dict(), os.path.join(output_dir, "deberta_v3_xsmall_probability_liar.pth"))


In [ ]:
# Ячейка 13: Тестирование результата (тестовые данные из датасета)

#model_bin.load_state_dict(torch.load(os.path.join(output_dir, "deberta_v3_xsmall_binary_liar.pth")))

checkpoint = torch.load(os.path.join(output_dir, "deberta_v3_xsmall_liar_binary_best_with_threshold.pth"), weights_only=False)
model_bin.load_state_dict(checkpoint['model_state_dict'])
truth_threshold = checkpoint['best_threshold']
print(f"Загрузили модель с порогом {truth_threshold:.3f}")



model_bin.eval()

test_preds = []
test_targets = []
test_probs = []

with torch.no_grad():
    for batch in tqdm(test_loader_bin, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch.get('labels')
        if labels is not None:
            labels = labels.to(device).float().unsqueeze(1)

        outputs = model_bin(input_ids, attention_mask)  # logits [batch_size, 1]
        probs = torch.sigmoid(outputs).cpu().numpy()
        preds = (probs > truth_threshold).astype(int).flatten()

        test_preds.extend(preds)
        test_probs.extend(probs.flatten())

        if labels is not None:
            test_targets.extend(labels.cpu().numpy().flatten())

if len(test_targets) > 0:
    test_targets = np.array(test_targets)
    test_preds = np.array(test_preds)
    print("Test Classification Report:")
    print(classification_report(test_targets, test_preds, digits=4, zero_division=0))
    print("Macro F1-score:", f1_score(test_targets, test_preds, average='macro'))
    print("Confusion Matrix:")
    print(confusion_matrix(test_targets, test_preds))
else:
    print("Тестовые метки отсутствуют. Выведены только предсказания.")

In [ ]:
import googletrans
print(googletrans.__version__)

In [ ]:
# Ячейка 14: Тестирование ручное
# Загрузите токенизатор, если ещё не загружен
tokenizer = AutoTokenizer.from_pretrained(config.MODEL)

model_bin = CustomModel(config.MODEL, config.NUM_CLASSES_BIN, 0.3).to(device)

checkpoint = torch.load(os.path.join(output_dir, "deberta_v3_xsmall_liar_binary_best_with_threshold.pth"), weights_only=False)
model_bin.load_state_dict(checkpoint['model_state_dict'])
truth_threshold = checkpoint['best_threshold']
print(f"Загрузили модель с порогом {truth_threshold:.3f}")

model_bin.eval()

def predict_text(text, threshold=truth_threshold):
    # Токенизация
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        logits = model_bin(input_ids, attention_mask)  # [batch_size, 1]
        probs = torch.sigmoid(logits).cpu().item()

    label = 1 if probs > threshold else 0
    label_name = "Правда" if label == 1 else "Ложь"
    return label_name, probs

print("Введите текст для классификации (пустая строка для выхода):")

from googletrans import Translator

translator = Translator()

def translate_text(text, dest_lang='en'):
    translation = translator.translate(text, dest=dest_lang)
    return translation.text

while True:
    raw_input = input(">>> ").strip()
    if raw_input == "":
        print("Выход.")
        break
    try:
        translated_text = translate_text(raw_input, dest_lang='en')
    except Exception as e:
        print(f"Ошибка перевода: {e}")
        continue

    print(f"Перевод: {translated_text}")
    pred_label, pred_prob = predict_text(translated_text, truth_threshold)
    print(f"Предсказание: {pred_label} (вероятность: {pred_prob:.4f})\n")
